# Feature Engineering with Just Latitude and Longitude

# 0. Env

In [1]:
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

# 1. Load Data

In [3]:
df = (
    pl.scan_csv("../data/AB_US_2023.csv")
    .select(pl.col(["latitude", "longitude", "price"]))
    .collect()
)